In [28]:
import requests
from bs4 import BeautifulSoup
import time
from collections import defaultdict

In [45]:
def crawl_page(number, dictionary, words) -> bool:
    # time.sleep(0.05) # sleep 50 milliseconds
    try: response = requests.get(f'https://www.bgjargon.com//last_added/{number}')
    except Exception as e: return False

    if response.status_code != 200: return False

    print("-" , end=" ")
    old_len = len(dictionary)

    for word_a in BeautifulSoup(response.content, "html.parser").select('a'):
        if word_a.text in words:
            continue # dont open links for words that are already crawled
        words.add(word_a.text)
        # time.sleep(0.05) # sleep 50 milliseconds

        try: response = requests.get(word_a['href'])
        except Exception as e: continue

        if response.status_code != 200:continue

        soup = BeautifulSoup(response.content, "html.parser")
        word = soup.find("h2").text
        for article in soup.find_all('article'):
            meaning = article.find('div', class_='meaning').find('p').text
            example = article.find('div', class_='example').find('p').text
            author = article.find('span', class_='author').find('a').text
            city = article.find('span', class_='date').text
            pos_vote = int(article.find('span', class_='vote_yes').text)
            neg_vote = int(article.find('span', class_='vote_no').text)
            dictionary[word]['meanings'].append({
                'meaning': meaning,
                'example': example,
                'author': author,
                'city': city,
                'pos_vote': pos_vote,
                'neg_vote': neg_vote
                })

        tags = soup.find('div', class_='tags')
        if tags != None:
            for tags_a in tags.find_all('a'):
                dictionary[word]['tags'].append(tags_a.text)

    return len(dictionary) > old_len

In [41]:
page_counter = 0
words = set()
dictionary = defaultdict(lambda: {'meanings': [], 'tags': []})

In [51]:
# if you get an exception just run it again, the results must have been saved
while(crawl_page(page_counter, dictionary, words)):
    page_counter += 1

print(f"Last page: {page_counter}")
print(f"Length of the dictionary: {len(dictionary)}")

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

KeyboardInterrupt: 

In [52]:
page_counter

2541

In [53]:
# saving to a file
import json
import codecs

json_object = json.dumps(dictionary, indent=4, ensure_ascii=False)
with codecs.open("data/bgjargon_all_meanings_tags.json", "w", "utf-8") as outfile:
    outfile.write(json_object)

json_object = json.dumps(list(words), indent=4, ensure_ascii=False)
with codecs.open("data/bgjargon_words.json", "w", "utf-8") as outfile:
    outfile.write(json_object)

In [54]:
# for loading from a file
import json

with open('data/bgjargon_all_meanings_tags.json', 'r', encoding="utf-8") as f:
    dictionary = json.load(f)

with open('data/bgjargon_words.json', 'r', encoding="utf-8") as f:
    words = json.load(f)
words = set(words)

In [58]:
page_counter = 2541

True